In [1]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

In [2]:
import os
from pathlib import Path

import keras
import numpy as np
import pandas as pd
import seaborn as sns
from keras import backend as K
from keras.optimizers import Adam
from matplotlib import pyplot as plt

import kfold_keras
from helper import (filter_guided, filter_lee, plot_training_history,
                    prepare_data)
from statoil_models import Larger_CNN, Simple_CNN

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

Using TensorFlow backend.


# CNN from scratch gridsearch
This notebook is a grid search for the best simple CNN model trained from scratch. Several parameters are varied:
- the design of the CNN either a small one with only 3 convolutional layers (32, 32,64 filters), or a larger one with four convolutional layers (16,32,64,128)
- number and nodes of the fully connected layer for each model can be selected using a width and depth parameter
- optional each model can have batch normalization layers 
- the filtering of the data is also varied: No pre-filtering, Guided or Lee Filter
- the scaling of the data is either: None or on a per image basis

The evaluation is done using a 5-fold Cross-validation with early stopping based on the loss on the validation set.
Only the image data is used. The incidence angle is disregarded.

** The best model uses the smaller CNN, a guided pre-filter, no data scaling and a fully connected layer with 128 nodes **
`Simple_CNN_128_gn3  Acc=86  , Epi=76  , ROC_AUC=0.95 , vloss=0.308`


## No Prefilter, no scaling

In [3]:
data_folder = Path("data")
train_file = data_folder / 'train.json'
test_file = data_folder / 'test.json'

train = pd.read_json(train_file) 
print('loaded training '+str(len(train)))
#test = pd.read_json(test_file) 
#print('loaded test '+str(len(train)))

y=train['is_iceberg'].values ## convert pandas series to numpy array
X=prepare_data(train,dim=[0,1,2],filter_function=None,rnd=False,scale=None)
input_shape= X[0].shape


loaded training 1604


In [4]:
grid_search= [
              (Simple_CNN(input_shape),'Simple_CNN_64'),
              (Simple_CNN(input_shape,width=2),'Simple_CNN_128'),
              (Simple_CNN(input_shape,width=4),'Simple_CNN_256'),
              (Simple_CNN(input_shape,depth=2),'Simple_CNN_2x64'),
              (Simple_CNN(input_shape,width=2,depth=2),'Simple_CNN_2x128'),
              (Simple_CNN(input_shape,width=4,depth=2),'Simple_CNN_2x256'),
              (Simple_CNN(input_shape,batch_norm=True),'Simple_CNN_bn64'),
              (Simple_CNN(input_shape,width=2,batch_norm=True),'Simple_CNN_bn128'),
              (Simple_CNN(input_shape,width=4,batch_norm=True),'Simple_CNN_bn256'),
              (Simple_CNN(input_shape,depth=2,batch_norm=True),'Simple_CNN_bn2x64'),
              (Simple_CNN(input_shape,width=2,depth=2,batch_norm=True),'Simple_CNN_bn2x128'),
              (Simple_CNN(input_shape,width=4,depth=2,batch_norm=True),'Simple_CNN_bn2x256')]
models=[]
results=[]
for model, name in grid_search:
    name += '_nn3'
    out = kfold_keras.k_fold_keras_early_stop(model,X,y,k=5,name=name,train_at_end=False,patience=20)
    models.append(name)
    results.append(out)

Simple_CNN_bn2x256_nn3: 100%|██████████| 5/5 [01:48<00:00, 20.42s/it, Acc=70.5, Epi=59.2, ROC_AUC=0.864, vloss=0.931]


## Guided Prefilter, no scaling


In [5]:
data_folder = Path("data")
train_file = data_folder / 'train.json'
test_file = data_folder / 'test.json'

train = pd.read_json(train_file) 
print('loaded training '+str(len(train)))
#test = pd.read_json(test_file) 
#print('loaded test '+str(len(train)))

y=train['is_iceberg'].values ## convert pandas series to numpy array
X=prepare_data(train,dim=[0,1,2],filter_function=filter_guided,rnd=False,scale=None)
input_shape= X[0].shape


loaded training 1604


In [6]:
grid_search= [
              (Simple_CNN(input_shape),'Simple_CNN_64'),
              (Simple_CNN(input_shape,width=2),'Simple_CNN_128'),
              (Simple_CNN(input_shape,width=4),'Simple_CNN_256'),
              (Simple_CNN(input_shape,depth=2),'Simple_CNN_2x64'),
              (Simple_CNN(input_shape,width=2,depth=2),'Simple_CNN_2x128'),
              (Simple_CNN(input_shape,width=4,depth=2),'Simple_CNN_2x256'),
              (Simple_CNN(input_shape,batch_norm=True),'Simple_CNN_bn64'),
              (Simple_CNN(input_shape,width=2,batch_norm=True),'Simple_CNN_bn128'),
              (Simple_CNN(input_shape,width=4,batch_norm=True),'Simple_CNN_bn256'),
              (Simple_CNN(input_shape,depth=2,batch_norm=True),'Simple_CNN_bn2x64'),
              (Simple_CNN(input_shape,width=2,depth=2,batch_norm=True),'Simple_CNN_bn2x128'),
              (Simple_CNN(input_shape,width=4,depth=2,batch_norm=True),'Simple_CNN_bn2x256')]
models=[]
results=[]
for model, name in grid_search:
    name += '_gn3'
    out = kfold_keras.k_fold_keras_early_stop(model,X,y,k=5,name=name,train_at_end=False,patience=20)
    models.append(name)
    results.append(out)

Simple_CNN_bn2x256_gn3: 100%|██████████| 5/5 [01:37<00:00, 20.27s/it, Acc=76.7, Epi=54.2, ROC_AUC=0.875, vloss=0.586]


## Lee Prefilter,  scaling

In [7]:
data_folder = Path("data")
train_file = data_folder / 'train.json'
test_file = data_folder / 'test.json'

train = pd.read_json(train_file) 
print('loaded training '+str(len(train)))
#test = pd.read_json(test_file) 
#print('loaded test '+str(len(train)))

y=train['is_iceberg'].values ## convert pandas series to numpy array
X=prepare_data(train,dim=[0,1,2],filter_function=filter_guided,rnd=False,scale='img')
input_shape= X[0].shape
print('filter: Lee scaled per image')

loaded training 1604
filter: Lee scaled per image


In [8]:
grid_search= [
              (Simple_CNN(input_shape),'Simple_CNN_64'),
              (Simple_CNN(input_shape,width=2),'Simple_CNN_128'),
              (Simple_CNN(input_shape,width=4),'Simple_CNN_256'),
              (Simple_CNN(input_shape,depth=2),'Simple_CNN_2x64'),
              (Simple_CNN(input_shape,width=2,depth=2),'Simple_CNN_2x128'),
              (Simple_CNN(input_shape,width=4,depth=2),'Simple_CNN_2x256'),
              (Simple_CNN(input_shape,batch_norm=True),'Simple_CNN_bn64'),
              (Simple_CNN(input_shape,width=2,batch_norm=True),'Simple_CNN_bn128'),
              (Simple_CNN(input_shape,width=4,batch_norm=True),'Simple_CNN_bn256'),
              (Simple_CNN(input_shape,depth=2,batch_norm=True),'Simple_CNN_bn2x64'),
              (Simple_CNN(input_shape,width=2,depth=2,batch_norm=True),'Simple_CNN_bn2x128'),
              (Simple_CNN(input_shape,width=4,depth=2,batch_norm=True),'Simple_CNN_bn2x256')]
models_lee=[]
results_lee=[]
for model, name in grid_search:
    name += '_li3'
    out = kfold_keras.k_fold_keras_early_stop(model,X,y,k=5,name=name,train_at_end=False,patience=20)
    models_lee.append(name)
    results_lee.append(out)
    



Simple_CNN_bn2x256_li3: 100%|██████████| 5/5 [01:45<00:00, 19.40s/it, Acc=73.7, Epi=44.4, ROC_AUC=0.875, vloss=0.629]


## Guided Prefilter, scaling

In [9]:
data_folder = Path("data")
train_file = data_folder / 'train.json'
test_file = data_folder / 'test.json'

train = pd.read_json(train_file) 
print('loaded training '+str(len(train)))
#test = pd.read_json(test_file) 
#print('loaded test '+str(len(train)))

y=train['is_iceberg'].values ## convert pandas series to numpy array
X=prepare_data(train,dim=[0,1,2],filter_function=filter_guided,rnd=False,scale='img')
input_shape= X[0].shape
print('filter: guided scaled')

loaded training 1604
filter: guided scaled


In [10]:
grid_search= [
              (Simple_CNN(input_shape),'Simple_CNN_64'),
              (Simple_CNN(input_shape,width=2),'Simple_CNN_128'),
              (Simple_CNN(input_shape,width=4),'Simple_CNN_256'),
              (Simple_CNN(input_shape,depth=2),'Simple_CNN_2x64'),
              (Simple_CNN(input_shape,width=2,depth=2),'Simple_CNN_2x128'),
              (Simple_CNN(input_shape,width=4,depth=2),'Simple_CNN_2x256'),
              (Simple_CNN(input_shape,batch_norm=True),'Simple_CNN_bn64'),
              (Simple_CNN(input_shape,width=2,batch_norm=True),'Simple_CNN_bn128'),
              (Simple_CNN(input_shape,width=4,batch_norm=True),'Simple_CNN_bn256'),
              (Simple_CNN(input_shape,depth=2,batch_norm=True),'Simple_CNN_bn2x64'),
              (Simple_CNN(input_shape,width=2,depth=2,batch_norm=True),'Simple_CNN_bn2x128'),
              (Simple_CNN(input_shape,width=4,depth=2,batch_norm=True),'Simple_CNN_bn2x256')]
models_gi3=[]
results_gi3=[]
for model, name in grid_search:
    name += '_gi3'
    out = kfold_keras.k_fold_keras_early_stop(model,X,y,k=5,name=name,train_at_end=False,patience=20)
    models_gi3.append(name)
    results_gi3.append(out)
    



Simple_CNN_bn2x256_gi3: 100%|██████████| 5/5 [01:56<00:00, 24.08s/it, Acc=77.2, Epi=50, ROC_AUC=0.863, vloss=0.679]  


## Guided Prefilter, no scaling, Larger_CNN model

In [11]:
data_folder = Path("data")
train_file = data_folder / 'train.json'
test_file = data_folder / 'test.json'

train = pd.read_json(train_file) 
print('loaded training '+str(len(train)))
#test = pd.read_json(test_file) 
#print('loaded test '+str(len(train)))

y=train['is_iceberg'].values ## convert pandas series to numpy array
X=prepare_data(train,dim=[0,1,2],filter_function=filter_guided,rnd=False,scale=None)
input_shape= X[0].shape
print('filter: guided')

loaded training 1604
filter: guided


In [12]:
grid_search= [
              (Larger_CNN(input_shape),'Larger_CNN_64'),
              (Larger_CNN(input_shape,width=2),'Larger_CNN_128'),
              (Larger_CNN(input_shape,width=4),'Larger_CNN_256'),
              (Larger_CNN(input_shape,depth=2),'Larger_CNN_2x64'),
              (Larger_CNN(input_shape,width=2,depth=2),'Larger_CNN_2x128'),
              (Larger_CNN(input_shape,width=4,depth=2),'Larger_CNN_2x256'),
              (Larger_CNN(input_shape,batch_norm=True),'Larger_CNN_bn64'),
              (Larger_CNN(input_shape,width=2,batch_norm=True),'Larger_CNN_bn128'),
              (Larger_CNN(input_shape,width=4,batch_norm=True),'Larger_CNN_bn256'),
              (Larger_CNN(input_shape,depth=2,batch_norm=True),'Larger_CNN_bn2x64'),
              (Larger_CNN(input_shape,width=2,depth=2,batch_norm=True),'Larger_CNN_bn2x128'),
              (Larger_CNN(input_shape,width=4,depth=2,batch_norm=True),'Larger_CNN_bn2x256')]
models_gn3_large=[]
results_gn3_large=[]
for model, name in grid_search:
    name += '_gn3_large'
    out = kfold_keras.k_fold_keras_early_stop(model,X,y,k=5,name=name,train_at_end=False,patience=20)
    models_gn3_large.append(name)
    results_gn3_large.append(out)
    



Larger_CNN_bn2x256_gn3_large: 100%|██████████| 5/5 [02:15<00:00, 26.81s/it, Acc=80, Epi=57.8, ROC_AUC=0.899, vloss=0.562]  


## Guided Prefilter, scaling, Larger_CNN model

In [13]:
data_folder = Path("data")
train_file = data_folder / 'train.json'
test_file = data_folder / 'test.json'

train = pd.read_json(train_file) 
print('loaded training '+str(len(train)))
#test = pd.read_json(test_file) 
#print('loaded test '+str(len(train)))

y=train['is_iceberg'].values ## convert pandas series to numpy array
X=prepare_data(train,dim=[0,1,2],filter_function=filter_guided,rnd=False,scale='img')
input_shape= X[0].shape
print('filter: guided')

loaded training 1604
filter: guided


In [14]:
grid_search= [
              (Larger_CNN(input_shape),'Larger_CNN_64'),
              (Larger_CNN(input_shape,width=2),'Larger_CNN_128'),
              (Larger_CNN(input_shape,width=4),'Larger_CNN_256'),
              (Larger_CNN(input_shape,depth=2),'Larger_CNN_2x64'),
              (Larger_CNN(input_shape,width=2,depth=2),'Larger_CNN_2x128'),
              (Larger_CNN(input_shape,width=4,depth=2),'Larger_CNN_2x256'),
              (Larger_CNN(input_shape,batch_norm=True),'Larger_CNN_bn64'),
              (Larger_CNN(input_shape,width=2,batch_norm=True),'Larger_CNN_bn128'),
              (Larger_CNN(input_shape,width=4,batch_norm=True),'Larger_CNN_bn256'),
              (Larger_CNN(input_shape,depth=2,batch_norm=True),'Larger_CNN_bn2x64'),
              (Larger_CNN(input_shape,width=2,depth=2,batch_norm=True),'Larger_CNN_bn2x128'),
              (Larger_CNN(input_shape,width=4,depth=2,batch_norm=True),'Larger_CNN_bn2x256')]
models_gn3_large=[]
results_gn3_large=[]
for model, name in grid_search:
    name += '_gn3_large'
    out = kfold_keras.k_fold_keras_early_stop(model,X,y,k=5,name=name,train_at_end=False,patience=20)
    models_gn3_large.append(name)
    results_gn3_large.append(out)
    



Larger_CNN_bn2x256_gn3_large: 100%|██████████| 5/5 [01:39<00:00, 20.25s/it, Acc=78.3, Epi=50.8, ROC_AUC=0.89, vloss=0.495] 


## No Prefilter, scaling, Larger_CNN model

In [15]:
data_folder = Path("data")
train_file = data_folder / 'train.json'
test_file = data_folder / 'test.json'

train = pd.read_json(train_file) 
print('loaded training '+str(len(train)))
#test = pd.read_json(test_file) 
#print('loaded test '+str(len(train)))

y=train['is_iceberg'].values ## convert pandas series to numpy array
X=prepare_data(train,dim=[0,1,2],filter_function=None,rnd=False,scale='img')
input_shape= X[0].shape


loaded training 1604


In [16]:
grid_search= [
              (Larger_CNN(input_shape),'Larger_CNN_64'),
              (Larger_CNN(input_shape,width=2),'Larger_CNN_128'),
              (Larger_CNN(input_shape,width=4),'Larger_CNN_256'),
              (Larger_CNN(input_shape,depth=2),'Larger_CNN_2x64'),
              (Larger_CNN(input_shape,width=2,depth=2),'Larger_CNN_2x128'),
              (Larger_CNN(input_shape,width=4,depth=2),'Larger_CNN_2x256'),
              (Larger_CNN(input_shape,batch_norm=True),'Larger_CNN_bn64'),
              (Larger_CNN(input_shape,width=2,batch_norm=True),'Larger_CNN_bn128'),
              (Larger_CNN(input_shape,width=4,batch_norm=True),'Larger_CNN_bn256'),
              (Larger_CNN(input_shape,depth=2,batch_norm=True),'Larger_CNN_bn2x64'),
              (Larger_CNN(input_shape,width=2,depth=2,batch_norm=True),'Larger_CNN_bn2x128'),
              (Larger_CNN(input_shape,width=4,depth=2,batch_norm=True),'Larger_CNN_bn2x256')]
models_gn3_large=[]
results_gn3_large=[]
for model, name in grid_search:
    name += '_gn3_large'
    out = kfold_keras.k_fold_keras_early_stop(model,X,y,k=5,name=name,train_at_end=False,patience=20)
    models_gn3_large.append(name)
    results_gn3_large.append(out)
    



Larger_CNN_bn2x256_gn3_large: 100%|██████████| 5/5 [01:54<00:00, 22.03s/it, Acc=77.2, Epi=57.4, ROC_AUC=0.898, vloss=0.621]


## No Prefilter, No scaling, Larger_CNN model

In [17]:
data_folder = Path("data")
train_file = data_folder / 'train.json'
test_file = data_folder / 'test.json'

train = pd.read_json(train_file) 
print('loaded training '+str(len(train)))
#test = pd.read_json(test_file) 
#print('loaded test '+str(len(train)))

y=train['is_iceberg'].values ## convert pandas series to numpy array
X=prepare_data(train,dim=[0,1,2],filter_function=None,rnd=False,scale=None)
input_shape= X[0].shape


loaded training 1604


In [18]:
grid_search= [
              (Larger_CNN(input_shape),'Larger_CNN_64'),
              (Larger_CNN(input_shape,width=2),'Larger_CNN_128'),
              (Larger_CNN(input_shape,width=4),'Larger_CNN_256'),
              (Larger_CNN(input_shape,depth=2),'Larger_CNN_2x64'),
              (Larger_CNN(input_shape,width=2,depth=2),'Larger_CNN_2x128'),
              (Larger_CNN(input_shape,width=4,depth=2),'Larger_CNN_2x256'),
              (Larger_CNN(input_shape,batch_norm=True),'Larger_CNN_bn64'),
              (Larger_CNN(input_shape,width=2,batch_norm=True),'Larger_CNN_bn128'),
              (Larger_CNN(input_shape,width=4,batch_norm=True),'Larger_CNN_bn256'),
              (Larger_CNN(input_shape,depth=2,batch_norm=True),'Larger_CNN_bn2x64'),
              (Larger_CNN(input_shape,width=2,depth=2,batch_norm=True),'Larger_CNN_bn2x128'),
              (Larger_CNN(input_shape,width=4,depth=2,batch_norm=True),'Larger_CNN_bn2x256')]
models_gn3_large=[]
results_gn3_large=[]
for model, name in grid_search:
    name += '_gn3_large'
    out = kfold_keras.k_fold_keras_early_stop(model,X,y,k=5,name=name,train_at_end=False,patience=20)
    models_gn3_large.append(name)
    results_gn3_large.append(out)
    



Larger_CNN_bn2x256_gn3_large: 100%|██████████| 5/5 [03:12<00:00, 36.03s/it, Acc=78.5, Epi=65, ROC_AUC=0.89, vloss=0.524]   
